In [1]:
import pandas as pd
from collections import Counter

df = pd.read_csv("transacoes_1.4.csv")

df

,carrinho,produto,descricao,categoria
0,380962,4059805069114,CAMISETA SUPERNOVA M RAWSTE M,RUNNING
1,380962,4055017073826,MEIA LINER CUSHION 3S 3P WHITE/WHITE/BLACK 3538,TRAINING
2,380962,4058032608264,CAMISETA MC FREELIFT PRIME GRNNIT M,TRAINING
3,380962,4059323533418,TENIS UltraBOOST All Terrain GRETHR/GRETWO/NOB...,RUNNING
4,329797,4057288560432,BERMUDA KNIT MGREYH/TRAGRE XL,ATHLETICS
...,...,...,...,...
1960181,5154858,4066746291269,JAQUETA SST TT PB BLACK MULTCO M,ORIGINALS
1960182,5120429,4065429048060,REGATA BRAND LOVE BLACK/WHITE S,TRAINING
1960183,5120429,4064055403434,CAMISETA OWN THE RUN HALMIN S,RUNNING
1960184,5120429,4065429171874,SHORTS 2 IN 1 BLACK/WHITE XS,TRAINING


In [2]:
#funçao que recebe o id de um produto + True e retorna os 3 itens que mais foram vendidos junto com esse produto
def relacionado(produto, singular):
    
    filtrado = df[df['produto'] == produto]
    carrinhos = filtrado['carrinho'].to_list()#value_counts()
    
    itens = []
    for i in carrinhos:
        filtro = df[df['carrinho'] == i]
        itens_c = filtro['produto'].to_list()
        for a in itens_c:
            itens.append(a)
    itens_contados = Counter(itens).most_common()
    for i in itens_contados:
        if i[0] == produto:
            itens_contados.remove(i)
    resultados = itens_contados[:3]
    if singular:
        print("melhores resultados", produto, ":")
        print(resultados)
        return resultados
    else:
        return itens_contados

#funçao que recebe um array de ids de produtos e retorna os 3 itens que mais foram vendidos junto com esses produtos somados
def relacionados(carrinho):
    itens = []
    for produto in carrinho:
        for i in relacionado(produto, False):
            for a in range(i[1]):
                itens.append(i[0])
                
    itens_contados = Counter(itens).most_common()
    for i in list(itens_contados):
        for j in carrinho:
            if i[0] == j and i in itens_contados:
                itens_contados.remove(i)
    resultados = itens_contados[:3]
    print("melhores resultados", carrinho, ":")
    print(resultados)
    return resultados

#recebe id do produto e retorna descrição(pode ser adaptada a base de dados de produtos no futuro)
def produto(id):
    if id == 4062062914349:
        return 'TENIS SUPERSTAR SLIP ON W CBLACK 35'
    if id == 4052549968248:
        return 'CARTEIRA GLAM preto NT'
    if id == 4064043251115:
        return 'MEIA LINER TREFOIL WHITE/WHITE/BLACK M'
    return df.loc[df['produto'] == id][:1]["descricao"].iloc[0]

#recebe uma descrição de um produto e retorna id do mesmo
def id_by_desc(desc):
    return df.loc[df['descricao'] == desc][:1]["produto"].iloc[0]

#verifica em casos excepcionais onde não há produtos suficientes vendidos com os produtos do carrinho e retorna a mesma lista de
#ids, adicionando os produtos mais vendidos caso necessario
def verifica(array):
    if len(array) == 3:
        return array
    elif len(array) == 2:
        array = array.append((4062062914349, 1))
    elif len(array) == 1:
        array = array.append((4062062914349, 1))
        array = array.append((4052549968248, 1))
    elif len(array) == 0:
        array = array.append((4062062914349, 1))
        array = array.append((4052549968248, 1))
        array = array.append((4064043251115, 1))
    else:
        return array
    return array

# função usada no streamlit com a base de dados de produtos improvisada(não tivemos acesso a base de produtos)
#def get_produtos():
#    return produtos['DESC'].values

In [32]:
#Teste para recomendação de produto unico - imprime 2x por que retorna o resultado

id_item = 4059323533418

relacionado(id_item, True)

melhores resultados 4059323533418 :
[(4057288272373, 3), (4059805069114, 2), (4055017073826, 1)]


[(4057288272373, 3), (4059805069114, 2), (4055017073826, 1)]

In [18]:
#Teste para recomendação de carrinho - imprime 2x por que retorna o resultado

id_itens = [4059805069114, 4055017073826, 4058032608264] 

relacionados(id_itens)

melhores resultados [4059805069114, 4055017073826, 4058032608264] :
[(4055017073819, 209), (4055017096481, 155), (4055017096504, 80)]


[(4055017073819, 209), (4055017096481, 155), (4055017096504, 80)]

## Execução de testes internos
#### testes de produtos separados

In [58]:
id_by_desc("CALCA ADI CHINO BLACK 3434")

4057286152486

In [59]:
produto(4057286152486)

'CALCA ADI CHINO BLACK 3434'

In [60]:
lista_desc = ['CALCA CHINO AEROTECH BLACK 3032',
 'CALCA CHINO PANTS BLACK 3032',
 'CALCA CHINO STRIPED HEMP/WHITE 3434']

for i in lista_desc:
    if produto(id_by_desc(i)) == i:
        print("True:", i)
    else:
        print("FALSE")
    print(id_by_desc(i))

True: CALCA CHINO AEROTECH BLACK 3032
4059322222115
True: CALCA CHINO PANTS BLACK 3032
4059807508413
True: CALCA CHINO STRIPED HEMP/WHITE 3434
4060512737692


In [76]:
produtos_sample = ['TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36', 'TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43', 'JAQUETA IVP VELOUR CHEWOO 2XS', 'TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36', 'TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38', 'CALCA ADI CHINO BLACK 3434', 'CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28', 'SHORTS VIAGEM SAO PAULO TEAMAR XL', 'CAMISETA OFICIAL EVENTO WC22 ACTMAR M', 'TENIS XPLR CBROWN/FTWWHT/TRACAR 40', 'TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34', 'SHORT RESPONSE M BLACK/BLACK S 7"', 'TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34', 'TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38', 'CAMISETA 4 0 LOG BLACK/WHITE XL', 'CALCA TIGHT 3 STRIPES DUSPNK L', 'SHORTS LONG BLACK L', 'BONE ADIDAS DE REDINHA BLACK/BLACK/WHITE OSFM', 'CAMISETA 3STRIPES LS WHITE/BLACK L', 'TENIS FORUM LOW FTWWHT/FTWWHT/CBLACK 37']

contador = 1
for i in produtos_sample:
    print(contador)
    print(i)
    for j in relacionado(id_by_desc(i), True):
        print(produto(j[0]))
    print("----------")
    contador+=1

1
TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36
melhores resultados 4064047720440 :
[(4064043251092, 3), (4064057711582, 2), (4062058558281, 2)]
MEIA LINER TREFOIL WHITE/WHITE/BLACK S
CAMISETA RYV CHSOGR M
MEIA LINER TREFOIL WHITE/BLACK/MGREYH 3942
----------
2
TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43
melhores resultados 4065426078343 :
[(4064057799139, 1)]
BERMUDA ESSENTIAL MGREYH XL
----------
3
JAQUETA IVP VELOUR CHEWOO 2XS
melhores resultados 4065431510456 :
[(4065431541320, 1), (4065431506572, 1), (4065431533509, 1)]
VESTIDO IVP SHOPNK M
BERMUDA IVP MENS CHEWOO/BLACK M
TOP IVP VEL BRALETT CHEWOO L
----------
4
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36
melhores resultados 4064047226980 :
[(4064043251092, 2), (4064056687970, 1), (4064053750073, 1)]
MEIA LINER TREFOIL WHITE/WHITE/BLACK S
BOLSA MINI AIRLINER MULTCO NT
TENIS 3MC SUPCOL 35
----------
5
TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38
melhores resultados 4059319528879 :
[(4055012755574, 2), (4059322699788, 1), (4060518495657, 1)]
MEIA CRE

In [66]:
lista_desc = ['TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36', 'TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43', 'JAQUETA IVP VELOUR CHEWOO 2XS', 'TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36', 'TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38', 'CALCA ADI CHINO BLACK 3434', 'CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28', 'SHORTS VIAGEM SAO PAULO TEAMAR XL', 'CAMISETA OFICIAL EVENTO WC22 ACTMAR M', 'TENIS XPLR CBROWN/FTWWHT/TRACAR 40', 'TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34', 'SHORT RESPONSE M BLACK/BLACK S 7"', 'TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34', 'TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38', 'CAMISETA 4 0 LOG BLACK/WHITE XL', 'CALCA TIGHT 3 STRIPES DUSPNK L', 'SHORTS LONG BLACK L', 'BONE ADIDAS DE REDINHA BLACK/BLACK/WHITE OSFM', 'CAMISETA 3STRIPES LS WHITE/BLACK L', 'TENIS FORUM LOW FTWWHT/FTWWHT/CBLACK 37']

contagem = 1
for i in lista_desc:
    if produto(id_by_desc(i)) != i:
        print("######### TEMOS UM PROBLEMA #############")
    print(contagem)
    print(i, ":")
    print(id_by_desc(i))
    print("----------")
    contagem+=1

1
TENIS STAN SMITH W FTWWHT/SUPCOL/CBURGU 36 :
4064047720440
----------
2
TENIS EQ21 FTWWHT/FTWWHT/PULBLU 43 :
4065426078343
----------
3
JAQUETA IVP VELOUR CHEWOO 2XS :
4065431510456
----------
4
TENIS SUPERSTAR W FTWWHT/FTWWHT/RAYPNK 36 :
4064047226980
----------
5
TENIS NMD R1 PK NGTCAR/NGTCAR/HIREGR 38 :
4059319528879
----------
6
CALCA ADI CHINO BLACK 3434 :
4057286152486
----------
7
CHUTEIRA NMZ MESSI 18 4 IN JR FTWWHT/CBLACK/FOOBLU 28 :
4059811561510
----------
8
SHORTS VIAGEM SAO PAULO TEAMAR XL :
4065423018182
----------
9
CAMISETA OFICIAL EVENTO WC22 ACTMAR M :
4065429101406
----------
10
TENIS XPLR CBROWN/FTWWHT/TRACAR 40 :
4059322522406
----------
11
TENIS SUPERSTAR W GREFIV/GREFIV/GRETHR 34 :
4058025377962
----------
12
SHORT RESPONSE M BLACK/BLACK S 7" :
4059322070686
----------
13
TENIS DEERUPT RUNNER PARLEY W FTWWHT/FTWWHT/BLUSPI 34 :
4059322642357
----------
14
TENIS ULTRABOOST OG W CBLACK/GRESIX/NGTFLA 38 :
4064036866517
----------
15
CAMISETA 4 0 LOG BLACK/WHITE XL 

#### testes baseados em carrinho

In [3]:
testes_carrinho = pd.read_csv("testes_carrinho.csv")
testes_carrinho

,carrinho,id_produtos,desc_produtos
0,1938144,"[4060515261064, 4059807341744, 4059811961976, ...","['SHORT CB SH SL LEGINK/GRETWO/SHOCYA M', 'TOP..."
1,3491421,"[4064054469653, 4060507958217, 4060507958132, ...","['SHORTS D2M SEASONAL BLACK/WHITE L', 'BERMUDA..."
2,3255963,"[4062065488564, 4062065434844, 4062055082451, ...","['MEIA FOLD CUFF CREW WHITE S', 'MEIA MID CUT ..."
3,589266,"[4060507801971, 4059322005596, 4059807879308, ...","['CAMISETA CAMO TREF TEE WHITE L', 'CAMISETA T..."
4,3356795,"[4061626844528, 4061626835946, 4062054919758, ...",['MEIA CANO BAIXO 3 PARES WHITE/WHITE/WHITE M'...
5,3575578,"[4064057794004, 4064048708973, 4064043251122]","['BOLSA FESTIVAL MONOGRAM BLACK/WHITE NT', 'BE..."
6,569397,"[4055344854990, 4056563522868, 4054075521591]","['TENIS SEELEY CONAVY/FTWWHT/CBLACK 41', 'TENI..."
7,2793765,"[4064043251092, 4062062914349]","['MEIA LINER TREFOIL WHITE/WHITE/BLACK S', 'TE..."
8,4391264,"[4065423688828, 4065432236553]","['SHORTS FARM BLACK S', 'TOP SUPORTE LEVE HYPE..."
9,2093812,"[4062056078330, 4062059489607]","['CALCA TP CARDBO XL', 'BONE CORDUROY MESA OSFM']"


In [16]:
ids_p = []
for i in testes_carrinho.id_produtos.to_list():
    r = relacionados(eval(i))
    id_p = [r[0][0],r[1][0],r[2][0]]
    ids_p.append(id_p)

melhores resultados [4060515261064, 4059807341744, 4059811961976, 4058027540555, 4060517697724, 4059807898163, 4059811765116, 4059322292132, 4060514204963, 4059807898170] :
[(4061626835946, 11), (4055342869637, 9), (4055342869644, 9)]
melhores resultados [4064054469653, 4060507958217, 4060507958132, 4064056830215, 4064047282887, 4064047282900, 4064056695661, 4062064850881] :
[(4060507945088, 72), (20000000000068, 65), (20000000000070, 53)]
melhores resultados [4062065488564, 4062065434844, 4062055082451, 4064044673602, 4064044845382, 4064044841452] :
[(4064044653475, 5), (4062065487185, 4), (4064055202464, 4)]
melhores resultados [4060507801971, 4059322005596, 4059807879308, 4060507978970, 4059326094787] :
[(4059322017117, 153), (4059322003165, 91), (4059322009297, 39)]
melhores resultados [4061626844528, 4061626835946, 4062054919758, 4061626844528] :
[(4061626809923, 368), (4061626844474, 157), (20000000000068, 95)]
melhores resultados [4064057794004, 4064048708973, 4064043251122] :
[

In [17]:
ids_p

[[4061626835946, 4055342869637, 4055342869644],
 [4060507945088, 20000000000068, 20000000000070],
 [4064044653475, 4062065487185, 4064055202464],
 [4059322017117, 4059322003165, 4059322009297],
 [4061626809923, 4061626844474, 20000000000068],
 [4064043251184, 4062058558281, 4064043251115],
 [4055017073819, 4055017096504, 4059323903402],
 [4064043251153, 4064043251115, 4062062914318],
 [4065417460454, 4064055403434, 4065423723406],
 [4062054626533, 4060507193335, 4062054770038]]

In [26]:
testes_carrinho2 = testes_carrinho
testes_carrinho2['ids_rec1'] = ids_p
testes_carrinho2

,carrinho,id_produtos,desc_produtos,ids,ids_rec1,primeira_rec_m1,segunda_rec_m1,terceira_rec_m1
0,1938144,"[4060515261064, 4059807341744, 4059811961976, ...","['SHORT CB SH SL LEGINK/GRETWO/SHOCYA M', 'TOP...","[4061626835946, 4055342869637, 4055342869644]","[4061626835946, 4055342869637, 4055342869644]",MEIA CANO MEDIO 3 PARES BLACK/BLACK/BLACK M,LETRA AVULSA MULTCO NT,NR INT CLUB MULTCO NT
1,3491421,"[4064054469653, 4060507958217, 4060507958132, ...","['SHORTS D2M SEASONAL BLACK/WHITE L', 'BERMUDA...","[4060507945088, 20000000000068, 20000000000070]","[4060507945088, 20000000000068, 20000000000070]",BERMUDA 3 STRIPES BLACK L,KIT FRESCOBOL -2,BARALHO PERSONALIZADO PAPEL CARTAO 300 GR -2
2,3255963,"[4062065488564, 4062065434844, 4062055082451, ...","['MEIA FOLD CUFF CREW WHITE S', 'MEIA MID CUT ...","[4064044653475, 4062065487185, 4064055202464]","[4064044653475, 4062065487185, 4064055202464]",CALCA ORIGINALS MULTCO M,MEIA TRICOLOR THIN WHITE M,REGATA CORSET WHITE S
3,589266,"[4060507801971, 4059322005596, 4059807879308, ...","['CAMISETA CAMO TREF TEE WHITE L', 'CAMISETA T...","[4059322017117, 4059322003165, 4059322009297]","[4059322017117, 4059322003165, 4059322009297]",CAMISETA TREFOIL WHITE L,CAMISETA TREFOIL MGREYH L,CAMISETA TREFOIL BLACK M
4,3356795,"[4061626844528, 4061626835946, 4062054919758, ...",['MEIA CANO BAIXO 3 PARES WHITE/WHITE/WHITE M'...,"[4061626809923, 4061626844474, 20000000000068]","[4061626809923, 4061626844474, 20000000000068]",MEIA CANO BAIXO 3 PARES BLACK/BLACK/BLACK M,MEIA CANO BAIXO 3 PARES WHITE/WHITE/WHITE S,KIT FRESCOBOL -2
5,3575578,"[4064057794004, 4064048708973, 4064043251122]","['BOLSA FESTIVAL MONOGRAM BLACK/WHITE NT', 'BE...","[4064043251184, 4062058558281, 4064043251115]","[4064043251184, 4062058558281, 4064043251115]",MEIA CREW SOLID BLACK/WHITE/WHITE M,MEIA LINER TREFOIL WHITE/BLACK/MGREYH 3942,MEIA LINER TREFOIL WHITE/WHITE/BLACK M
6,569397,"[4055344854990, 4056563522868, 4054075521591]","['TENIS SEELEY CONAVY/FTWWHT/CBLACK 41', 'TENI...","[4055017073819, 4055017096504, 4059323903402]","[4055017073819, 4055017096504, 4059323903402]",MEIA LINER CUSHION 3S 3P WHITE/WHITE/BLACK 3942,MEIA LINER CUSHION 3S 3P BLACK/BLACK/WHITE 3942,MEIA NO SHOW 3PP WHITE/BLACK 3942
7,2793765,"[4064043251092, 4062062914349]","['MEIA LINER TREFOIL WHITE/WHITE/BLACK S', 'TE...","[4064043251153, 4064043251115, 4062062914318]","[4064043251153, 4064043251115, 4062062914318]",MEIA LINER TREFOIL BLACK/BLACK/WHITE S,MEIA LINER TREFOIL WHITE/WHITE/BLACK M,TENIS SUPERSTAR SLIP ON W CBLACK 36
8,4391264,"[4065423688828, 4065432236553]","['SHORTS FARM BLACK S', 'TOP SUPORTE LEVE HYPE...","[4065417460454, 4064055403434, 4065423723406]","[4065417460454, 4064055403434, 4065423723406]",TOP FARM LIGHT SUPORTE BLACK/PRINT MAC,CAMISETA OWN THE RUN HALMIN S,REGATA FARM BLACK/STPANU M
9,2093812,"[4062056078330, 4062059489607]","['CALCA TP CARDBO XL', 'BONE CORDUROY MESA OSFM']","[4062054626533, 4060507193335, 4062054770038]","[4062054626533, 4060507193335, 4062054770038]",CALCA RYV SP BLACK XL,CALCA TREFOIL BLACK XL,CALCA RYV SP CWHITE XL


In [32]:
primeira_rec_m1 = []
segunda_rec_m1 = []
terceira_rec_m1 = []
for i in ids_p:
    primeira_rec_m1.append(produto(i[0]))
    segunda_rec_m1.append(produto(i[1]))
    terceira_rec_m1.append(produto(i[2]))
testes_carrinho2['primeira_rec_m1'] = primeira_rec_m1
testes_carrinho2['segunda_rec_m1'] = segunda_rec_m1
testes_carrinho2['terceira_rec_m1'] = terceira_rec_m1
testes_carrinho2

,carrinho,id_produtos,desc_produtos,ids_rec1,primeira_rec_m1,segunda_rec_m1,terceira_rec_m1
0,1938144,"[4060515261064, 4059807341744, 4059811961976, ...","['SHORT CB SH SL LEGINK/GRETWO/SHOCYA M', 'TOP...","[4061626835946, 4055342869637, 4055342869644]",MEIA CANO MEDIO 3 PARES BLACK/BLACK/BLACK M,LETRA AVULSA MULTCO NT,NR INT CLUB MULTCO NT
1,3491421,"[4064054469653, 4060507958217, 4060507958132, ...","['SHORTS D2M SEASONAL BLACK/WHITE L', 'BERMUDA...","[4060507945088, 20000000000068, 20000000000070]",BERMUDA 3 STRIPES BLACK L,KIT FRESCOBOL -2,BARALHO PERSONALIZADO PAPEL CARTAO 300 GR -2
2,3255963,"[4062065488564, 4062065434844, 4062055082451, ...","['MEIA FOLD CUFF CREW WHITE S', 'MEIA MID CUT ...","[4064044653475, 4062065487185, 4064055202464]",CALCA ORIGINALS MULTCO M,MEIA TRICOLOR THIN WHITE M,REGATA CORSET WHITE S
3,589266,"[4060507801971, 4059322005596, 4059807879308, ...","['CAMISETA CAMO TREF TEE WHITE L', 'CAMISETA T...","[4059322017117, 4059322003165, 4059322009297]",CAMISETA TREFOIL WHITE L,CAMISETA TREFOIL MGREYH L,CAMISETA TREFOIL BLACK M
4,3356795,"[4061626844528, 4061626835946, 4062054919758, ...",['MEIA CANO BAIXO 3 PARES WHITE/WHITE/WHITE M'...,"[4061626809923, 4061626844474, 20000000000068]",MEIA CANO BAIXO 3 PARES BLACK/BLACK/BLACK M,MEIA CANO BAIXO 3 PARES WHITE/WHITE/WHITE S,KIT FRESCOBOL -2
5,3575578,"[4064057794004, 4064048708973, 4064043251122]","['BOLSA FESTIVAL MONOGRAM BLACK/WHITE NT', 'BE...","[4064043251184, 4062058558281, 4064043251115]",MEIA CREW SOLID BLACK/WHITE/WHITE M,MEIA LINER TREFOIL WHITE/BLACK/MGREYH 3942,MEIA LINER TREFOIL WHITE/WHITE/BLACK M
6,569397,"[4055344854990, 4056563522868, 4054075521591]","['TENIS SEELEY CONAVY/FTWWHT/CBLACK 41', 'TENI...","[4055017073819, 4055017096504, 4059323903402]",MEIA LINER CUSHION 3S 3P WHITE/WHITE/BLACK 3942,MEIA LINER CUSHION 3S 3P BLACK/BLACK/WHITE 3942,MEIA NO SHOW 3PP WHITE/BLACK 3942
7,2793765,"[4064043251092, 4062062914349]","['MEIA LINER TREFOIL WHITE/WHITE/BLACK S', 'TE...","[4064043251153, 4064043251115, 4062062914318]",MEIA LINER TREFOIL BLACK/BLACK/WHITE S,MEIA LINER TREFOIL WHITE/WHITE/BLACK M,TENIS SUPERSTAR SLIP ON W CBLACK 36
8,4391264,"[4065423688828, 4065432236553]","['SHORTS FARM BLACK S', 'TOP SUPORTE LEVE HYPE...","[4065417460454, 4064055403434, 4065423723406]",TOP FARM LIGHT SUPORTE BLACK/PRINT MAC,CAMISETA OWN THE RUN HALMIN S,REGATA FARM BLACK/STPANU M
9,2093812,"[4062056078330, 4062059489607]","['CALCA TP CARDBO XL', 'BONE CORDUROY MESA OSFM']","[4062054626533, 4060507193335, 4062054770038]",CALCA RYV SP BLACK XL,CALCA TREFOIL BLACK XL,CALCA RYV SP CWHITE XL


In [35]:
testes_carrinho2.to_csv("testes_carrinho_1.1.csv", index=False)

In [34]:
testes_carrinho2.to_excel("testes_carrinho_excel_1.1.xlsx")

In [147]:
%%time
relacionado(4061626844528, True)

melhores resultados 4061626844528 :
[(4061626809923, 179), (4061626844474, 73), (20000000000068, 45)]
Wall time: 6.44 s


[(4061626809923, 179), (4061626844474, 73), (20000000000068, 45)]

In [6]:
%%time
relacionado(4062059489607, True)

melhores resultados 4062059489607 :
[(4062059489652, 2), (4064041896868, 2), (4059807747263, 2)]
Wall time: 160 ms


[(4062059489652, 2), (4064041896868, 2), (4059807747263, 2)]